In [63]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from nltk.classify import ClassifierI
import numpy as np

## Importing 

In [64]:
nyt = pd.read_csv('../3. Sentiment Analysis preparation/sentiment_prepared.csv')

In [65]:
x = [i.replace("'", '') for i in nyt.tokens[0].replace('[', '').replace(']', '').split(',')]

In [67]:
nyt.dtypes

Unnamed: 0           int64
articleWordCount     int64
headline            object
snippet             object
tokens              object
label_headline      object
dtype: object

In [68]:
nyt.head()

,Unnamed: 0,articleWordCount,headline,snippet,tokens,label_headline
0,0,1324,G.O.P. Leadership Poised to Topple Obama’s Pi...,The most powerful and ambitious Republican-led...,"['leadership', 'poised', 'topple', 'obama', 'p...",Positive
1,1,2836,Fractured World Tested the Hope of a Young Pre...,A strategy that went from a “good war” to the ...,"['fractured', 'world', 'tested', 'hope', 'youn...",Positive
2,2,445,Little Troublemakers,Chuck Deodene puts us in a bubbly mood.,"['little', 'troublemakerschuck', 'deodene', 'p...",Neutral
3,3,864,"Angela Merkel, Russia’s Next Target","With a friend entering the White House, Vladim...","['angela', 'merkel', 'russia', 'next', 'target...",Negative
4,4,309,Boots for a Stranger on a Bus,Witnessing an act of generosity on a rainy day.,"['boots', 'stranger', 'buswitnessing', 'act', ...",Neutral


## New dataframe to eliminate neutral words

In [69]:
new_nyt = nyt[nyt.label_headline != 'Neutral']

In [70]:
new_nyt.replace(['Positive', 'Negative'], ['pos', 'neg'])
        

,Unnamed: 0,articleWordCount,headline,snippet,tokens,label_headline
0,0,1324,G.O.P. Leadership Poised to Topple Obama’s Pi...,The most powerful and ambitious Republican-led...,"['leadership', 'poised', 'topple', 'obama', 'p...",pos
1,1,2836,Fractured World Tested the Hope of a Young Pre...,A strategy that went from a “good war” to the ...,"['fractured', 'world', 'tested', 'hope', 'youn...",pos
3,3,864,"Angela Merkel, Russia’s Next Target","With a friend entering the White House, Vladim...","['angela', 'merkel', 'russia', 'next', 'target...",neg
7,7,557,Sprint and Mr. Trump’s Fictional Jobs,"The emerging, and dangerous, new form of crony...","['sprint', 'mr', 'trump', 'fictional', 'jobsth...",neg
9,9,1109,"Fighting Diabetes, and Leading by Example","Eric L. Adams, the Brooklyn borough president,...","['fighting', 'diabetes', 'leading', 'exampleer...",pos
...,...,...,...,...,...,...
7659,7659,3074,"The Questions, With Context",The questions show the special counsel’s focus...,"['questions', 'contextthe', 'questions', 'show...",neg
7661,7661,562,Watch Out for the Obstacles,Jacob Stulberg points out the speed bumps in h...,"['watch', 'obstaclesjacob', 'stulberg', 'point...",neg
7662,7662,1766,A Competitive Team Sport. For Your Dog.,"Flyball, in which four-dog teams compete in a ...","['competitive', 'team', 'sport', 'dogflyball',...",pos
7666,7666,659,Gen. Michael Hayden Has One Regret: Russia,"The former N.S.A. and C.I.A. chief on Trump, S...","['gen', 'michael', 'hayden', 'one', 'regret', ...",neg


In [71]:
def row_strip (ser):
    for row in ser:
        row = row.rstrip()
        return(row)
    
all_words = new_nyt.tokens.apply(row_strip)

In [72]:
all_words = nltk.FreqDist(all_words)

In [73]:
most_common = all_words.most_common(1000)

In [74]:
most_common_words = []

for word in most_common:
    w = word[0]
    most_common_words.append(w)

In [75]:
featuresets = new_nyt[['tokens', 'label_headline']].apply(lambda x: ({i: i in most_common_words for i in x[0]}, x[1]), axis=1)

In [77]:
featuresets

0       ({'[': True, ''': False, 'l': False, 'e': Fals...
1       ({'[': True, ''': False, 'f': False, 'r': Fals...
3       ({'[': True, ''': False, 'a': False, 'n': Fals...
7       ({'[': True, ''': False, 's': False, 'p': Fals...
9       ({'[': True, ''': False, 'f': False, 'i': Fals...
                              ...                        
7659    ({'[': True, ''': False, 'q': False, 'u': Fals...
7661    ({'[': True, ''': False, 'w': False, 'a': Fals...
7662    ({'[': True, ''': False, 'c': False, 'o': Fals...
7666    ({'[': True, ''': False, 'g': False, 'e': Fals...
7667    ({'[': True, ''': False, 'n': False, 'o': Fals...
Length: 5279, dtype: object

In [55]:
training_set = featuresets[:4000]
testing_set = featuresets[4000:]

In [56]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [57]:
print("Naive Bayes accuracy percent:", 
      (nltk.classify.accuracy(classifier, testing_set))*100)

Naive Bayes accuracy percent: 51.52462861610634


In [81]:
def clean_up(s):
    s = re.sub('[^A-Za-z]+', ' ', s)
    return s.lower().strip()

In [82]:
def tokenize(s):
    s = s.split(' ')
    return s

In [83]:
nyt['tokens'] = nyt['tokens'].apply(clean_up)

In [84]:
nyt['tokens'] = nyt['tokens'].apply(tokenize)

In [8]:
def row_strip (ser):
    for row in ser:
        row = row.rstrip()
        return(row)
    
all_words = nyt.tokens.apply(row_strip)

In [9]:
list_of_words = {}

for word in all_words:
    if word not in list_of_words.keys():
        list_of_words[word] = 1
    else:
        list_of_words[word] += 1

In [10]:
all_words = nltk.FreqDist(all_words)

In [11]:
most_common = all_words.most_common(1000)

In [31]:
most_common_words = []

for word in most_common:
    w = word[0]
    most_common_words.append(w)

In [33]:
most_common_words

['trump',
 'new',
 'teaching',
 'questions',
 'mr',
 'president',
 'variety',
 'going',
 'good',
 'one',
 'donald',
 'big',
 'democrats',
 'white',
 'women',
 'china',
 'house',
 'friday',
 'americans',
 'little',
 'homes',
 'walking',
 'america',
 'mindful',
 'republicans',
 'let',
 'love',
 'billions',
 'two',
 'life',
 'health',
 'first',
 'get',
 'britain',
 'eat',
 'think',
 'black',
 'end',
 'great',
 'real',
 'republican',
 'tax',
 'would',
 'facebook',
 'world',
 'taking',
 'stephen',
 'like',
 'time',
 'season',
 'million',
 'right',
 'obama',
 'better',
 'homeland',
 'heart',
 'north',
 'former',
 'finding',
 'student',
 'supreme',
 'much',
 'years',
 'state',
 'want',
 'another',
 'alienist',
 'rupaul',
 'fighting',
 'year',
 'yes',
 'looking',
 'old',
 'high',
 'art',
 'making',
 'dear',
 'jimmy',
 'obamacare',
 'case',
 'long',
 'feud',
 'man',
 'help',
 'trade',
 'trevor',
 'metoo',
 'heck',
 'pregnancy',
 'star',
 'family',
 'back',
 'american',
 'parents',
 'times',
 'f

In [66]:
# nyt.tokens.apply(lambda x: [i.replace("'", '') for i in x.replace('[', '').replace(']', '').split(',')])

0       [leadership,  poised,  topple,  obama,  pillar...
1       [fractured,  world,  tested,  hope,  young,  p...
2       [little,  troublemakerschuck,  deodene,  puts,...
3       [angela,  merkel,  russia,  next,  targetwith,...
4       [boots,  stranger,  buswitnessing,  act,  gene...
                              ...                        
7663    [dear,  match,  book,  fiction,  takes,  us,  ...
7664    [common,  question,  reinforces,  gender,  pay...
7665    [anna,  llama,  methe,  beginning,  middle,  e...
7666    [gen,  michael,  hayden,  one,  regret,  russi...
7667    [nothin,  like,  tunein,  something,  wonderfu...
Name: tokens, Length: 7668, dtype: object